In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '953cf27b-79db-4d43-9883-939fafe349ba',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print("true")

In [ ]:
import pandas as pd

In [ ]:
pd.set_option("display.max.columns",37)
pd.set_option("display.max_rows", 12)
df=pd.json_normalize(data["data"])
df['timestamp']=pd.to_datetime('now')
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '953cf27b-79db-4d43-9883-939fafe349ba',
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      # print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
       print(e)
    pd.set_option("display.max.columns",37)
    pd.set_option("display.max_rows", 15)
    df2=pd.json_normalize(data["data"])
    df2['timestamp']=pd.to_datetime('now')
    df=pd.concat([df,df2])
    if not os.path.isfile(r'C:\Users\alaak\OneDrive\Bureau\jupyter\APIS\API.csv'):
        df.to_csv(r'C:\Users\alaak\OneDrive\Bureau\jupyter\APIS\API.csv', header=True)
    else:
        df.to_csv(r'C:\Users\alaak\OneDrive\Bureau\jupyter\APIS\API.csv', mode='a', header =False)
        

In [ ]:
import os 
from time import time 
from time import sleep
for i in range(333):
    api_runner()
    print('API runner completed successfully')
    sleep(60) #sleep for one minute
exit()

In [ ]:
df3=pd.read_csv(r"C:\Users\alaak\OneDrive\Bureau\jupyter\APIS\API.csv")
df3


In [ ]:
pd.set_option('display.float.format',lambda x: '%.5f' %x)

In [ ]:
df3

In [ ]:
df4=df3.groupby('name',sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d',]].mean()

In [ ]:
df5=df4.stack()
pd.set_option('display.max.rows',None)
print(df5)

In [ ]:
df6=df5.to_frame(name='values')

In [ ]:
# index=pd.Index(range(75))
# df6
df6.reset_index(inplace=True)
df6.rename(columns={'level_1':'percent change'},inplace=True)
df6

In [ ]:
df6['percent change'] = df6['percent change'].str.replace('quote.USD.percent_change_', '', regex=False)
df6[df6['values']<0]

In [ ]:
df6['values'] = pd.to_numeric(df6['values'], errors='coerce')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(150, 35))
sns.catplot(x='percent change', y='values', hue='name', data=df6, kind='point')
plt.show()

In [ ]:
pd.set_option('display.max.columns',38)
df3.head(5)
df7=df3[['name','quote.USD.price','timestamp']]
df_bitcoin=df7[df7['name']=='Bitcoin']
df_bitcoin=df_bitcoin.drop_duplicates(subset='quote.USD.price')
df_bitcoin['timestamp'] = pd.to_datetime(df_bitcoin['timestamp'])
df_bitcoin['timestamp']=df_bitcoin['timestamp'].dt.strftime('%H:%M')

In [ ]:
sns.set_theme(style='darkgrid')
sns.lineplot(x='timestamp',y='quote.USD.price',data=df_bitcoin)
plt.ylim(df_bitcoin['quote.USD.price'].min() - 1, df_bitcoin['quote.USD.price'].max() + 1)